In [ ]:
# In this notebook I remove the incorrect data (the ones with weird dates) and the wrong quality/felix data.
# I probably won't need this again (the quality/felix removal part) since Iara did it manually shortly after.

In [1]:
import pandas as pd
from datetime import datetime
import re

In [2]:
df = pd.read_csv('../data/absorbance/raw.csv')

# extracting the year, so that we can remove the wrong data

In [3]:
regex = '^\w+.\w+,\d{2}/\d{2}/(\d{4})$'

In [4]:
df['year'] = df[df.columns.values[0]].apply(lambda x: re.findall(regex, x)[0])

# dropping data with wrong date

In [5]:
rows_to_drop = df[df['year'] == '1601']
df.drop(rows_to_drop.index, inplace=True)

In [6]:
df.reset_index(drop=True, inplace=True)

# getting shelf life and treatment info

In [7]:
regex = '^\w*(T\d)(B\d)(\d)'

In [8]:
df['TREATMENT'] = df['Filename,'].apply(lambda x: re.findall(regex, x)[0][0])
df['BLOCK'] = df['Filename,'].apply(lambda x: re.findall(regex, x)[0][1])
df['SHELF-LIFE'] = df['Filename,'].apply(lambda x: re.findall(regex, x)[0][2])

# removing unnecessary info

In [9]:
cols = list(df.columns.values[:2])
cols.append('year')

In [10]:
df.drop(cols, axis=1, inplace=True)

# saving new data

In [11]:
df.to_csv('../data/absorbance/treated.csv')

# removing wrong felix data

In [77]:
# Removing these bc in the felix data, there are 20 samples for B1 and 24 for B3, whereas in the quality .csv
# there are 42 samples in total. Assuming that the 24 B3 samples are correct (since this should be the exact
# value), there should be only 18 samples for B1. Thus, I'm removing the last two from B1
df[df['TREATMENT'] == 'T6'][df['SHELF-LIFE'] == '1'][df['BLOCK']=='B1'].tail(2)

/home/danielesreis/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,285,288,291,294,297,300,303,306,309,312,...,1182,1185,1188,1191,1194,1197,1200,TREATMENT,BLOCK,SHELF-LIFE
660,0,0,0,0,0,0.0,0,0,0.0,0,...,0,0,0,0,0,0,0,T6,B1,1
661,0,0,0,0,0,0.0,0,0,0.0,0,...,0,0,0,0,0,0,0,T6,B1,1


In [25]:
qlt[qlt['FIRMNESS'] == 10][qlt['TREATMENT'] == 'T6'][qlt['SHELF-LIFE'] == '1']

/home/danielesreis/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,SHELF-LIFE,TREATMENT,NUM,FRESH MASS,L,C,H,FIRMNESS,LP,CP,HP,SST,TOTAL ACIDITY,DRY MASS
368,1,T6,11,506.02,69.63,52.93,80.45,10.0,69.08,68.45,82.12,10.9,NaN,17.43
387,1,T6,30,572.65,70.91,45.16,77.36,10.0,63.01,66.21,76.99,16.6,0.84,19.96


In [27]:
qlt[qlt['DRY MASS'] == 20.51]

,SHELF-LIFE,TREATMENT,NUM,FRESH MASS,L,C,H,FIRMNESS,LP,CP,HP,SST,TOTAL ACIDITY,DRY MASS
492,1,T4,23,513.28,65.47,52.12,62.2,6.9,59.83,64.26,79.27,16.5,NaN,20.51
765,3,T3,32,554.05,66.14,36.83,87.2,52.0,70.01,61.08,80.53,13.3,0.77,20.51


In [29]:
qlt.iloc[492:494]

,SHELF-LIFE,TREATMENT,NUM,FRESH MASS,L,C,H,FIRMNESS,LP,CP,HP,SST,TOTAL ACIDITY,DRY MASS
492,1,T4,23,513.28,65.47,52.12,62.20,6.9,59.83,64.26,79.27,16.5,NaN,20.51
493,1,T4,24,513.28,71.87,56.41,78.29,13.8,66.32,67.88,79.67,16.2,NaN,19.98


In [26]:
qlt.iloc[366:370]

,SHELF-LIFE,TREATMENT,NUM,FRESH MASS,L,C,H,FIRMNESS,LP,CP,HP,SST,TOTAL ACIDITY,DRY MASS
366,1,T6,9,584.40,59.95,53.83,53.55,13.8,53.15,51.41,78.85,14.7,NaN,18.35
367,1,T6,10,584.40,67.81,54.19,76.87,14.0,68.53,70.37,80.48,14.2,NaN,21.26
368,1,T6,11,506.02,69.63,52.93,80.45,10.0,69.08,68.45,82.12,10.9,NaN,17.43
369,1,T6,12,506.02,64.17,50.81,64.60,8.0,69.47,66.59,80.64,15.7,NaN,17.74


In [78]:
df.drop([660, 661], inplace=True)
df.reset_index(drop=True, inplace=True)

In [79]:
df.to_csv('../data/absorbance/treated_wo_incorrect.csv')

# appending the quality data

In [12]:
qlt = pd.read_csv('../data/quality/raw.csv')

In [13]:
qlt.drop(columns=['LEVELS'], inplace=True)
qlt['TREATMENT'] = qlt['TREATMENT'].apply(lambda x: str(x))
qlt['SHELF-LIFE'] = qlt['SHELF-LIFE'].apply(lambda x: str(x))

# removing wrong quality data

In [82]:
# removing from quality data bc there are 46 samples from felix and 48 from quality .csv. B3 is supposedly correct,
# because it contains 24 felix samples, whereas there's 22 B1 samples. So, I'm gonna remove the last two from B1.
qlt[qlt['TREATMENT']=='T4'][qlt['SHELF-LIFE']=='1'].iloc[22:24]

/home/danielesreis/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


,SHELF-LIFE,TREATMENT,NUM,FRESH MASS,L,C,H,FIRMNESS,LP,CP,HP,SST,TOTAL ACIDITY,DRY MASS
492,1,T4,23,513.28,65.47,52.12,62.20,6.9,59.83,64.26,79.27,16.5,NaN,20.51
493,1,T4,24,513.28,71.87,56.41,78.29,13.8,66.32,67.88,79.67,16.2,NaN,19.98


In [83]:
qlt.drop([492, 493], inplace=True)
qlt.reset_index(drop=True, inplace=True)

In [84]:
qlt.to_csv('../data/quality/treated_wo_incorrect.csv')

# indexing by treatment and shelf-life to allow assignment

In [113]:
new_df = df.sort_values(by=['TREATMENT', 'SHELF-LIFE']).reset_index(drop=True)

In [114]:
new_qlt = qlt.sort_values(by=['TREATMENT', 'SHELF-LIFE']).reset_index(drop=True)

In [115]:
attrs = list(new_qlt.columns[3:].values)
for att in attrs:
    new_df[att] = new_qlt[att]

In [119]:
new_df.shape

(828, 320)

In [118]:
new_df.columns[1:307]

Index(['288', '291', '294', '297', '300', '303', '306', '309', '312', '315',
       ...
       '1176', '1179', '1182', '1185', '1188', '1191', '1194', '1197', '1200',
       'TREATMENT'],
      dtype='object', length=306)

In [104]:
cols = ['TREATMENT', 'BLOCK', 'SHELF-LIFE'] + attrs + list(new_df.columns[1:307].values)

In [111]:
new_df = new_df[cols]

KeyError: "['TREATMENT'] not in index"

In [109]:
new_df.set_index('TREATMENT', inplace=True)

In [27]:
new_df.to_csv('../data/final/data_final.csv')